In [1]:
# importing dependencies
import numpy
import sys
import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Using TensorFlow backend.


In [0]:
# loading data
file = open("84-0.txt").read()

In [0]:
# tokenzing
# standardizing
def tokenize_words(input):
  input = input.lower()
  tokenizer = RegexpTokenizer(r'\w+')
  tokens = tokenizer.tokenize(input)
  filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
  return "".join(filtered)

processed_inputs = tokenize_words(file)  

  

In [0]:
# chars to number
chars = sorted(list(set(processed_inputs)))
char_to_num = dict((c,i) for i, c in enumerate(chars))

In [5]:
# check if words to char and chars to num has worked
input_len = len(processed_inputs)
vocab_len = len(chars)
print("The total number of character: " , input_len)
print("Total vocab: " , vocab_len)

The total number of character:  233296
Total vocab:  42


In [0]:
# seq length
seq_length = 100
x_data = []
y_data = []

In [7]:
# loop through the sequence
for i in range (0, input_len - seq_length, 1):
  in_seq = processed_inputs[i:i + seq_length]
  out_seq = processed_inputs[i + seq_length]
  x_data.append([char_to_num[char] for char in in_seq])
  y_data.append(char_to_num[out_seq])

n_patterns = len(x_data)
print ("Total patterns: ", n_patterns)  

Total patterns:  233196


In [0]:
# convert input sequence to np array and so on
X = numpy.reshape(x_data, (n_patterns, seq_length, 1))
X = X/float(vocab_len)

In [0]:
# one-hot encoding
y = np_utils.to_categorical(y_data)

In [0]:
# creating the model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [0]:
# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
# saving weights
filepath = "model_weights_saved.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

In [13]:
# fit model and let it train
model.fit(X,y, epochs=4, batch_size=256, callbacks=desired_callbacks)

Epoch 1/4
233196/233196 [==============================] - 414s 2ms/step - loss: 2.9360

Epoch 00001: loss improved from inf to 2.93596, saving model to model_weights_saved.hdf5
Epoch 2/4
233196/233196 [==============================] - 391s 2ms/step - loss: 2.9160

Epoch 00002: loss improved from 2.93596 to 2.91600, saving model to model_weights_saved.hdf5
Epoch 3/4
233196/233196 [==============================] - 389s 2ms/step - loss: 2.9133

Epoch 00003: loss improved from 2.91600 to 2.91328, saving model to model_weights_saved.hdf5
Epoch 4/4
233196/233196 [==============================] - 388s 2ms/step - loss: 2.8947

Epoch 00004: loss improved from 2.91328 to 2.89470, saving model to model_weights_saved.hdf5


In [0]:
# recompile model with saved weights
filename = "model_weights_saved.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')


In [0]:
# output of the model back into characters
num_to_char = dict((i, c) for i, c in enumerate (chars))


In [21]:
# random seed to help generate
start = numpy.random.randint(0, len(x_data)-1)
pattern = x_data[start]
print("Random Seed: ")
print("\"", ''.join([num_to_char[value] for value in pattern]), "\"")

Random Seed: 
" avegreatestpleasurereceiveletteruncledatedparislongerformidabledistancemayhopeseelessfortnightpoorco "


In [22]:
# generate the text
for i in range (100):
  x = numpy.reshape(pattern, (1,len(pattern), 1))
  x = x/float(vocab_len)
  prediction = model.predict(x, verbose =0)
  index = numpy.argmax(prediction)
  result = num_to_char[index]
  seq_in = [num_to_char[value] for value in pattern]
  sys.stdout.write(result)
  pattern.append(index)
  pattern = pattern[1:len(pattern)] 

eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee